## Chapter 1.2: Data Structure and Manipulation
***
In the previous tutorial we saw how we can create a model by calling a few lines of code from imported data. However modelling tends to be an iterative process. Here we will explore the tools that `GemPy` with the help of `pandas` offers to modify the input data of a model.

There are 5 main  funtion "types" in GemPy:

- *create*: 
    - create new objects
    - return the objects
 
- *set*
    - set given values **inplace**

- *update*
    - update dataframe or other attribute from other object or many objects. Usually this object is not passed as argument (this is the main difference with map)

- *map*
    - update dataframe (so far mainly df) or other attribute from an object to another object.
    - Completelly directed. One attribute/property is updated by another one.
    - In general, we map_from so the method is in the mutated object.

- *get*
    - return an image of the object
    
The intention is that a function/method that does not fall in any of these categories has a name (verb in principle) self explanatory. 


As always we start importing the usual packages and reading expample data:

In [84]:
# These two lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../..")

# Importing GemPy
import gempy as gp

# Embedding matplotlib figures in the notebooks
%matplotlib inline

# Importing auxiliary libraries
import numpy as np
import pandas as pn
import matplotlib.pyplot as plt

## Series

Series is the object that contains the properties associated with each independent scalar field. Right now it is simply the order of the series (which is infered by the index order). But in the future will be add the unconformity relation or perhaps the type of interpolator

Series and Faults classes are quite entagled since fauls are a type of series

In [2]:
series = gp.Series(faults)
series.df

,order_series,BottomRelation
Default series,1,Erosion


We can modify the series bt using `set_series_index`:

In [3]:
series.set_series_index(['foo', 'foo2', 'foo5', 'foo7'])
series

,order_series,BottomRelation
foo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion


The index of series are pandas categories. These provides quite handy backend functionality (see pandas.Categorical).

In [4]:
series.df.index

CategoricalIndex(['foo', 'foo2', 'foo5', 'foo7'], categories=['foo', 'foo2', 'foo5', 'foo7'], ordered=False, dtype='category')

For adding new series:

In [5]:
series.add_series('foo3')
series

,order_series,BottomRelation
foo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion
foo3,5,Erosion


Delete series

In [6]:
series.delete_series('foo3')
series

,order_series,BottomRelation
foo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion


Rename series:

In [7]:
series.rename_series({'foo':'boo'})
series

,order_series,BottomRelation
boo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion


Reorder series:

In [8]:
series.reorder_series(['foo2', 'boo', 'foo7', 'foo5'])
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion


### Faults

The *df faults* is used to charectirize which *mathematical series* behave as fault and if mentioned faults are finite or infinite. Both df should get updated automatically as we modify the series object linked to the fault object (by passing it wehn a Series object is created).

In [9]:
faults = gp.Faults()
faults

,isFault,isFinite
foo2,False,False
boo,False,False
foo7,False,False
foo5,False,False


Finally we have the *faults relations df* which captures which *mathematical series* a given fault offset in order to reproduce complex faulting networks

In [10]:
faults.faults_relations_df

,foo2,boo,foo7,foo5
foo2,False,False,False,False
boo,False,False,False,False
foo7,False,False,False,False
foo5,False,False,False,False


We can use `set_is_fault` to choose which of our series are faults:

In [11]:
faults.set_is_fault(['boo'])

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False


Similar thing for the fault relations:

In [12]:
fr = np.zeros((4, 4))
fr[2, 2] = True
faults.set_fault_relation(fr)

,foo2,boo,foo7,foo5
foo2,False,False,False,False
boo,False,False,False,False
foo7,False,False,False,False
foo5,False,False,False,False


Now if we change the series df and we update the series already defined will conserve their values while the new ones will be set to false:

In [13]:
series.add_series('foo20')


In [14]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


In [15]:
faults

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


In [16]:
faults.faults_relations_df

,foo2,boo,foo7,foo5,foo20
foo2,False,False,False,False,False
boo,False,False,False,False,False
foo7,False,False,False,False,False
foo5,False,False,False,False,False
foo20,False,False,False,False,False


When we add new series the values switch  to NaN. We will be careful not having any nan in the DataFrames or we will raise errors down the line.

In [17]:
faults.set_is_fault()

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


In [18]:
faults.set_fault_relation()

,foo2,boo,foo7,foo5,foo20
foo2,False,False,False,False,False
boo,False,False,False,False,False
foo7,False,False,False,False,False
foo5,False,False,False,False,False
foo20,False,False,False,False,False


### Surfaces:

The *df* surfaces contains three properties. *id* refers to the order of the surfaces on the sequential pile, i.e. the strict order of computation. *values* on the other hand is the final value that each voxel will have after discretization. This may be useful for example in the case we want to map a specific geophysical property (such as density) to a given unity. By default both are the same since to discretize lithological units the value is arbitrary.

#### From an empty df

The Surfaces class needs to have an associate series object. This will limit the name of the series since they are a pandas.Category

In [19]:
surfaces = gp.Surfaces(series)

We can set any number of formations by passing a list with the names. By default they will take the name or the first series.

In [20]:
surfaces.set_surfaces_names(['foo', 'foo2', 'foo5'])


True

In [21]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


We can add new formations:

In [22]:
surfaces.add_surface(['feeeee'])
surfaces

,surface,series,order_surfaces,isBasement,color,id
0,foo,foo20,1,False,#015482,1
1,foo2,foo20,2,False,#9f0052,2
2,foo5,foo20,3,False,#ffbe00,3
3,feeeee,foo20,4,True,#728f02,4


The column formation is also a pandas.Categories. This will be important for the Data clases (surface_points and Orientations)

In [23]:
surfaces.df['surface']

0       foo
1      foo2
2      foo5
3    feeeee
Name: surface, dtype: object

In [24]:
surfaces

,surface,series,order_surfaces,isBasement,color,id
0,foo,foo20,1,False,#015482,1
1,foo2,foo20,2,False,#9f0052,2
2,foo5,foo20,3,False,#ffbe00,3
3,feeeee,foo20,4,True,#728f02,4


### Set values

To set the values we do it with the following method

In [25]:
surfaces.set_surfaces_values([2,2,2,5])

,surface,series,order_surfaces,isBasement,color,id,value_0
0,foo,foo20,1,False,#015482,1,2
1,foo2,foo20,2,False,#9f0052,2,2
2,foo5,foo20,3,False,#ffbe00,3,2
3,feeeee,foo20,4,True,#728f02,4,5


In [26]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,value_0
0,foo,foo20,1,False,#015482,1,2
1,foo2,foo20,2,False,#9f0052,2,2
2,foo5,foo20,3,False,#ffbe00,3,2
3,feeeee,foo20,4,True,#728f02,4,5


#### Set values with a given name:

We can give specific names to the properties (i.e. density)

In [27]:
surfaces.add_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])

,surface,series,order_surfaces,isBasement,color,id,value_0,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,2,1
3,feeeee,foo20,4,True,#728f02,4,5,6,8


In [28]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,value_0,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,2,1
3,feeeee,foo20,4,True,#728f02,4,5,6,8


### Delete formations values

To delete a full propery:

In [29]:
surfaces.delete_surface_values(['val_foo', 'value_0'])

True

#### One of the formations must be set be the basement:

In [30]:
surfaces.set_basement()
surfaces

,surface,series,order_surfaces,isBasement,color,id,val2_foo
0,foo,foo20,1,False,#015482,1,2
1,foo2,foo20,2,False,#9f0052,2,2
2,foo5,foo20,3,False,#ffbe00,3,1
3,feeeee,foo20,4,True,#728f02,4,8


#### Set formation values

We can also use set values instead adding. This will delete the previous properties and add the new one

In [31]:
surfaces.set_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,1
3,feeeee,foo20,4,True,#728f02,4,6,8


The last property is the correspondant series that each formation belong to. `series` and `formation` are pandas categories. To get a overview of what this mean check https://pandas.pydata.org/pandas-docs/stable/categorical.html.

In [32]:
surfaces.df['series']

0    foo20
1    foo20
2    foo20
3    foo20
Name: series, dtype: category
Categories (5, object): [foo2, boo, foo7, foo5, foo20]

In [33]:
surfaces.df['surface']

0       foo
1      foo2
2      foo5
3    feeeee
Name: surface, dtype: object

### Map series to formation

To map a series to a formation we can do it by passing a dict:

In [34]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,1
3,feeeee,foo20,4,True,#728f02,4,6,8


In [35]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


If a series does not exist in the `Series` object, we rise a warning and we set those formations to nans

In [36]:
d =  {"foo7":'foo', "booX": ('foo2','foo5', 'fee')}

In [37]:
surfaces.map_series(d)

In [38]:
surfaces.map_series({"foo7":'foo', "boo": ('foo2','foo5', 'fee')})

In [39]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,foo2,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#ffbe00,2,2,1
0,foo,foo7,1,False,#015482,3,2,2
3,feeeee,foo20,1,True,#728f02,4,6,8


An advantage of categories is that they are order so no we can tidy the df by series and formation

### Modify surface name

In [40]:
surfaces.rename_surfaces({'foo2':'lala'})

True

In [41]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,lala,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#ffbe00,2,2,1
0,foo,foo7,1,False,#015482,3,2,2
3,feeeee,foo20,1,True,#728f02,4,6,8


In [42]:
surfaces.df.loc[2, 'val_foo'] = 22

In [43]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,lala,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#ffbe00,2,22,1
0,foo,foo7,1,False,#015482,3,2,2
3,feeeee,foo20,1,True,#728f02,4,6,8


### Modify surface color

The surfaces DataFrame also contains a column for the color in which the surfaces are displayed. To change the color, call 

In [44]:
surfaces.colors.change_colors()

Click to select new colors.


This allow to change the colors interactively. If you already know which colors you want to use, you can also update them with a dictionary mapping the surface name to a hex color string:

In [45]:
new_colors={'foo':'#ff8000', 'foo5':'#4741be'}
surfaces.colors.change_colors(new_colors)

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,lala,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#4741be,2,22,1
0,foo,foo7,1,False,#ff8000,3,2,2
3,feeeee,foo20,1,True,#728f02,4,6,8


# Data
#### surface_points
These two DataFrames (df from now on) will contain the individual information of each point at an interface or orientation. Some properties of this table are mapped from the *df* below.

In [46]:
surface_points = gp.SurfacePoints(surfaces)
#orientations = gp.Orientations()

In [47]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series


In [48]:
surface_points.set_surface_points(pn.DataFrame(np.random.rand(6,3)), ['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [49]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.220326,0.184814,0.969042,NaN,NaN,NaN,foo,NaN,NaN,1
1,0.164605,0.891807,0.631484,NaN,NaN,NaN,foo5,NaN,NaN,1
2,0.467465,0.408204,0.837860,NaN,NaN,NaN,lala,NaN,NaN,1
3,0.918191,0.334664,0.421819,NaN,NaN,NaN,foo5,NaN,NaN,1
4,0.161164,0.580530,0.422838,NaN,NaN,NaN,lala,NaN,NaN,1
5,0.806562,0.520184,0.910495,NaN,NaN,NaN,feeeee,NaN,NaN,1


In [50]:
surface_points.map_data_from_surfaces(surfaces, 'series')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.220326,0.184814,0.969042,NaN,NaN,NaN,foo,foo7,NaN,1
1,0.164605,0.891807,0.631484,NaN,NaN,NaN,foo5,boo,NaN,1
2,0.467465,0.408204,0.837860,NaN,NaN,NaN,lala,boo,NaN,1
3,0.918191,0.334664,0.421819,NaN,NaN,NaN,foo5,boo,NaN,1
4,0.161164,0.580530,0.422838,NaN,NaN,NaN,lala,boo,NaN,1
5,0.806562,0.520184,0.910495,NaN,NaN,NaN,feeeee,foo20,NaN,1


In [51]:
surface_points.map_data_from_surfaces(surfaces, 'id')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.220326,0.184814,0.969042,NaN,NaN,NaN,foo,foo7,3,1
1,0.164605,0.891807,0.631484,NaN,NaN,NaN,foo5,boo,2,1
2,0.467465,0.408204,0.837860,NaN,NaN,NaN,lala,boo,1,1
3,0.918191,0.334664,0.421819,NaN,NaN,NaN,foo5,boo,2,1
4,0.161164,0.580530,0.422838,NaN,NaN,NaN,lala,boo,1,1
5,0.806562,0.520184,0.910495,NaN,NaN,NaN,feeeee,foo20,4,1


In [52]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


In [53]:
surface_points.map_data_from_series(series, 'order_series')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.220326,0.184814,0.969042,NaN,NaN,NaN,foo,foo7,3,3
1,0.164605,0.891807,0.631484,NaN,NaN,NaN,foo5,boo,2,2
2,0.467465,0.408204,0.837860,NaN,NaN,NaN,lala,boo,1,2
3,0.918191,0.334664,0.421819,NaN,NaN,NaN,foo5,boo,2,2
4,0.161164,0.580530,0.422838,NaN,NaN,NaN,lala,boo,1,2
5,0.806562,0.520184,0.910495,NaN,NaN,NaN,feeeee,foo20,4,5


In [54]:
surface_points.sort_table()
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
2,0.467465,0.408204,0.837860,NaN,NaN,NaN,lala,boo,1,2
4,0.161164,0.580530,0.422838,NaN,NaN,NaN,lala,boo,1,2
1,0.164605,0.891807,0.631484,NaN,NaN,NaN,foo5,boo,2,2
3,0.918191,0.334664,0.421819,NaN,NaN,NaN,foo5,boo,2,2
0,0.220326,0.184814,0.969042,NaN,NaN,NaN,foo,foo7,3,3
5,0.806562,0.520184,0.910495,NaN,NaN,NaN,feeeee,foo20,4,5


In [55]:
faults

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


### Orientations

In [56]:
orientations = gp.Orientations(surfaces)

In [57]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series


### Set values passing pole vectors:

In [58]:
orientations.set_orientations(np.random.rand(6,3)*10,
                            np.random.rand(6,3),
                            surface=['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [59]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,7.945031,9.742227,1.840072,NaN,NaN,NaN,0.160542,0.875893,0.722958,43.700723,10.386442,1,foo,NaN,NaN,1
1,9.321850,5.022950,6.512587,NaN,NaN,NaN,0.883693,0.221993,0.867743,29.802595,75.898489,1,foo5,NaN,NaN,1
2,1.304196,6.984417,2.355604,NaN,NaN,NaN,0.552697,0.871022,0.694725,45.994685,32.396689,1,lala,NaN,NaN,1
3,0.331830,7.297069,7.807146,NaN,NaN,NaN,0.205127,0.028796,0.853366,31.420284,82.009017,1,foo5,NaN,NaN,1
4,0.817948,2.250443,2.242857,NaN,NaN,NaN,0.824398,0.282608,0.011465,89.343078,71.078036,1,lala,NaN,NaN,1
5,8.352581,1.555950,6.213878,NaN,NaN,NaN,0.174670,0.590878,0.998350,3.291803,16.468272,1,feeeee,NaN,NaN,1


### Set values pasing orientation data: azimuth, dip, pole (dip direction)

In [60]:
orientations.set_orientations(np.random.rand(6,3)*10,
                            orientation = np.random.rand(6,3)*20,
                            surface=['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [61]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,3.410937,9.263962,9.133879,NaN,NaN,NaN,0.075150,0.457697,3.780294,6.994973,9.324270,3.808642,foo,NaN,NaN,1
1,2.714327,9.084130,9.805760,NaN,NaN,NaN,0.033977,0.384508,3.880002,5.681443,5.049772,3.899156,foo5,NaN,NaN,1
2,1.486381,6.126908,1.908434,NaN,NaN,NaN,0.001061,0.062433,8.550553,0.418406,0.973626,8.550781,lala,NaN,NaN,1
3,5.880151,9.931062,6.225713,NaN,NaN,NaN,0.921052,5.566677,16.940149,18.421687,9.394939,17.855108,foo5,NaN,NaN,1
4,9.102183,6.204672,6.588491,NaN,NaN,NaN,0.079623,0.386906,1.525309,14.519105,11.628733,1.575628,lala,NaN,NaN,1
5,3.059328,5.962268,5.995935,NaN,NaN,NaN,0.004835,0.302054,6.308774,2.741486,0.917123,6.316003,feeeee,NaN,NaN,1


### Mapping data from the other df

In [62]:
orientations.map_data_from_surfaces(surfaces, 'series')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,3.410937,9.263962,9.133879,NaN,NaN,NaN,0.075150,0.457697,3.780294,6.994973,9.324270,3.808642,foo,foo7,NaN,1
1,2.714327,9.084130,9.805760,NaN,NaN,NaN,0.033977,0.384508,3.880002,5.681443,5.049772,3.899156,foo5,boo,NaN,1
2,1.486381,6.126908,1.908434,NaN,NaN,NaN,0.001061,0.062433,8.550553,0.418406,0.973626,8.550781,lala,boo,NaN,1
3,5.880151,9.931062,6.225713,NaN,NaN,NaN,0.921052,5.566677,16.940149,18.421687,9.394939,17.855108,foo5,boo,NaN,1
4,9.102183,6.204672,6.588491,NaN,NaN,NaN,0.079623,0.386906,1.525309,14.519105,11.628733,1.575628,lala,boo,NaN,1
5,3.059328,5.962268,5.995935,NaN,NaN,NaN,0.004835,0.302054,6.308774,2.741486,0.917123,6.316003,feeeee,foo20,NaN,1


In [63]:
orientations.map_data_from_surfaces(surfaces, 'id')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,3.410937,9.263962,9.133879,NaN,NaN,NaN,0.075150,0.457697,3.780294,6.994973,9.324270,3.808642,foo,foo7,3,1
1,2.714327,9.084130,9.805760,NaN,NaN,NaN,0.033977,0.384508,3.880002,5.681443,5.049772,3.899156,foo5,boo,2,1
2,1.486381,6.126908,1.908434,NaN,NaN,NaN,0.001061,0.062433,8.550553,0.418406,0.973626,8.550781,lala,boo,1,1
3,5.880151,9.931062,6.225713,NaN,NaN,NaN,0.921052,5.566677,16.940149,18.421687,9.394939,17.855108,foo5,boo,2,1
4,9.102183,6.204672,6.588491,NaN,NaN,NaN,0.079623,0.386906,1.525309,14.519105,11.628733,1.575628,lala,boo,1,1
5,3.059328,5.962268,5.995935,NaN,NaN,NaN,0.004835,0.302054,6.308774,2.741486,0.917123,6.316003,feeeee,foo20,4,1


In [64]:
orientations.map_data_from_series(series, 'order_series')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,3.410937,9.263962,9.133879,NaN,NaN,NaN,0.075150,0.457697,3.780294,6.994973,9.324270,3.808642,foo,foo7,3,3
1,2.714327,9.084130,9.805760,NaN,NaN,NaN,0.033977,0.384508,3.880002,5.681443,5.049772,3.899156,foo5,boo,2,2
2,1.486381,6.126908,1.908434,NaN,NaN,NaN,0.001061,0.062433,8.550553,0.418406,0.973626,8.550781,lala,boo,1,2
3,5.880151,9.931062,6.225713,NaN,NaN,NaN,0.921052,5.566677,16.940149,18.421687,9.394939,17.855108,foo5,boo,2,2
4,9.102183,6.204672,6.588491,NaN,NaN,NaN,0.079623,0.386906,1.525309,14.519105,11.628733,1.575628,lala,boo,1,2
5,3.059328,5.962268,5.995935,NaN,NaN,NaN,0.004835,0.302054,6.308774,2.741486,0.917123,6.316003,feeeee,foo20,4,5


In [65]:
orientations.update_annotations()

### Grid

In [66]:
grid = gp.Grid()
grid.set_regular_grid([0,10,0,10,0,10], [50,50,50])

In [67]:
grid.values

array([[ 0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.3],
       [ 0.1,  0.1,  0.5],
       ..., 
       [ 9.9,  9.9,  9.5],
       [ 9.9,  9.9,  9.7],
       [ 9.9,  9.9,  9.9]])

#### Rescaling Data

In [68]:
rescaling = gp.RescaledData(surface_points, orientations, grid)


In [69]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
2,0.467465,0.408204,0.837860,0.286469,0.261560,0.280737,lala,boo,1,2
4,0.161164,0.580530,0.422838,0.270755,0.270401,0.259446,lala,boo,1,2
1,0.164605,0.891807,0.631484,0.270931,0.286370,0.270150,foo5,boo,2,2
3,0.918191,0.334664,0.421819,0.309592,0.257788,0.259393,foo5,boo,2,2
0,0.220326,0.184814,0.969042,0.273790,0.250100,0.287467,foo,foo7,3,3
5,0.806562,0.520184,0.910495,0.303865,0.267305,0.284463,feeeee,foo20,4,5


In [70]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,annotations
0,3.410937,9.263962,9.133879,0.437474,0.715877,0.706338,0.075150,0.457697,3.780294,6.994973,9.324270,3.808642,foo,foo7,3,3,"${\bf{x}}_{\beta \,{\bf{3}},0}$"
1,2.714327,9.084130,9.805760,0.401737,0.706651,0.740807,0.033977,0.384508,3.880002,5.681443,5.049772,3.899156,foo5,boo,2,2,"${\bf{x}}_{\beta \,{\bf{2}},0}$"
2,1.486381,6.126908,1.908434,0.338741,0.554940,0.335660,0.001061,0.062433,8.550553,0.418406,0.973626,8.550781,lala,boo,1,2,"${\bf{x}}_{\beta \,{\bf{1}},0}$"
3,5.880151,9.931062,6.225713,0.564149,0.750100,0.557144,0.921052,5.566677,16.940149,18.421687,9.394939,17.855108,foo5,boo,2,2,"${\bf{x}}_{\beta \,{\bf{2}},1}$"
4,9.102183,6.204672,6.588491,0.729445,0.558929,0.575755,0.079623,0.386906,1.525309,14.519105,11.628733,1.575628,lala,boo,1,2,"${\bf{x}}_{\beta \,{\bf{1}},1}$"
5,3.059328,5.962268,5.995935,0.419436,0.546494,0.545356,0.004835,0.302054,6.308774,2.741486,0.917123,6.316003,feeeee,foo20,4,5,"${\bf{x}}_{\beta \,{\bf{4}},0}$"


### Additional Data

In [81]:
ad = gp.AdditionalData(surface_points, orientations, grid, faults, surfaces, rescaling)


In [72]:
ad

values
Structure isLith                                                                True
          isFault                                                               True
          number faults                                                            1
          number surfaces                                                          4
          number series                                                            3
          number surfaces per series                                       [2, 1, 1]
          len surfaces surface_points                                   [2, 2, 1, 1]
          len series surface_points                                        [4, 1, 1]
          len series orientations                                          [4, 1, 1]
Options   dtype                                                              float64
          output                                                             geology
          theano_optimizer                                              fast_compile
          device                                                                 cpu
          verbosity                                                               []
Kriging   range                                                              1732.05
          $C_o$                                                              71428.6
          drift equations                                                  [3, 0, 0]
          nugget grad                                                           0.01
          nugget scalar                                                        1e-06
Rescaling rescaling factor                                                   19.4925
          centers                      [4.63167323509, 5.05793831164, 5.11378942781]

In [73]:
ad.structure_data

,values
isLith,True
isFault,True
number faults,1
number surfaces,4
number series,3
number surfaces per series,"[2, 1, 1]"
len surfaces surface_points,"[2, 2, 1, 1]"
len series surface_points,"[4, 1, 1]"
len series orientations,"[4, 1, 1]"


In [74]:
ad.options

,values
dtype,float64
output,geology
theano_optimizer,fast_compile
device,cpu
verbosity,[]


In [75]:
ad.options.df

,dtype,output,theano_optimizer,device,verbosity
values,float64,geology,fast_compile,cpu,[]


In [76]:
ad.options.df.dtypes

dtype               category
output              category
theano_optimizer    category
device              category
verbosity             object
dtype: object

In [77]:
ad.kriging_data

,values
range,1732.05
$C_o$,71428.6
drift equations,"[3, 0, 0]"
nugget grad,0.01
nugget scalar,1e-06


In [78]:
ad.rescaling_data

,values
rescaling factor,19.4925
centers,"[4.63167323509, 5.05793831164, 5.11378942781]"


### Interpolator

In [82]:
interp = gp.InterpolatorModel(surface_points, orientations, grid, surfaces, series, faults, ad)

In [83]:
interp.compile_th_fn()

Compiling theano function...
Compilation Done!


In [ ]:
interp.print_theano_shared()